In [2]:
#get data on ways fighter won/lost through dec, won/lost through sub, won/lost through ko/tko  
#tabulate outcomes in a table such as: Outcome, % chance, winner 
#FRONT END: give the user the ability to input two fighters (drop down menu)
# ---- FINISHED --- 
#extra: prevent users from picking two fighters that are from different weight classes
#--- LONG RUN --- 
# Improve model where it takes # of strikes (successful, attempted) , ground game/submissions duration, takedowns (successful, attempted)
# into consideration 

In [3]:
#get accuracy of model by spitting data into train/test
import pandas as pd 

In [10]:
df = pd.read_csv("UFC Matches April 2023.csv", )

In [11]:
df.head()

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
0,Tom Aaron,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
1,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Matt Ricehouse,LOSS,U-DEC,3.0,5:00
2,Tom Aaron,WEIGHT: 155 lbs.,Tom Aaron\n \n Eric Steenberg,WIN,SUB,1.0,0:56
3,Danny Abbadi,WEIGHT: 155 lbs.,NaN,NaN,NaN,NaN,NaN
4,Danny Abbadi,WEIGHT: 155 lbs.,Danny Abbadi\n \n Jorge Gurgel,LOSS,S-DEC,3.0,5:00


## Data Cleaning

We have to clean our dataset then group each fighter and get their distributions to determine their likelihood of winning/losing via decision, tko/ko or submission.

In [58]:
# delete first row of every fighter
nulls = df[["Matchup", "Outcome", 'Method', 'Round', 'Time']].isnull().all(axis=1) 
df = df.loc[~nulls, :]

#find matches that are upcoming and delete them
df = df[df["Outcome"] != 'NEXT']

# delete "weight text" in weight column
df['Weight'] = df['Weight'].str.replace('WEIGHT: ', '')

# keep only opponents name in matchup column
def remove_name(orig_matchup):
    last_index = orig_matchup.rfind('\n')
    new_string = orig_matchup[last_index +1:]
    return new_string.strip()

df['Matchup'] = df['Matchup'].apply(lambda x: remove_name(x))

In [61]:
df

,Full Name,Weight,Matchup,Outcome,Method,Round,Time
1,Tom Aaron,155 lbs.,Matt Ricehouse,LOSS,U-DEC,3.0,5:00
2,Tom Aaron,155 lbs.,Eric Steenberg,WIN,SUB,1.0,0:56
4,Danny Abbadi,155 lbs.,Jorge Gurgel,LOSS,S-DEC,3.0,5:00
5,Danny Abbadi,155 lbs.,Kalib Starnes,LOSS,SUB,1.0,2:56
7,Nariman Abbasov,155 lbs.,Ismael Bonfim,LOSS,U-DEC,3.0,5:00
...,...,...,...,...,...,...,...
22976,Alex Zuniga,145 lbs.,Bryan Caraway,LOSS,U-DEC,3.0,5:00
22979,Allan Zuniga,155 lbs.,John Gunther,LOSS,M-DEC,3.0,5:00
22981,Virgil Zwicker,205 lbs.,Guto Inocente,LOSS,U-DEC,3.0,5:00
22982,Virgil Zwicker,205 lbs.,Brett Albee,WIN,KO/TKO,1.0,1:46


## Exploratory Data Analysis (EDA)

In [62]:
#questions that could be of importance:
#what is the most common way of winning?
#for fighters that win through ko/submission what round does it typically occur in?
#is there a discrepancy in the way fighters win across different weight classes?